In [1]:
import pickle

with open("counterfactuals/wiki_Meta-Llama-3-8B->chat_llama3_prompt:first_k_sents:500_prompt_first_k:5_max_new_tokens:25.pkl", "rb") as f:
    data = pickle.load(f)


In [13]:
data["original"][67], data["counter"][67]

original = data["original"]
counter = data["counter"]

with open("out.txt", "w") as f:
    for o,c in zip(original,counter):
        print(o["text"])
        f.write(o["text"] + "\n")
        print("---------------------------")
        print(c["text"])
        f.write(c["text"] + "\n")
        f.write("================================================\n")
        print("=================================================")

The film stars M. G. Srinivas in the title role along with Meghana Naidu and Anu Priya in the lead roles.
Watch the
---------------------------
The film stars M. G. Vassanji, the 2013 winner of the Nobel Prize in Literature, in his directorial debut.
Toronto-based
Naarda plenirena is a species of Lepidopteran moth of the family NOCTUIDAE, found primarily in Southern Sri Lanka. Very small in
---------------------------
Naarda plenirena is a species of snout moth in the genus Naarda. It was described by Francis Walker in 1863. It is found in
Sponsored by the American Federation of Musicians of the United States and Canada (AFM)
This event is free for current AFM members!
Not a
---------------------------
Sponsored by the American Federation of Labor and Congress of Industrial Organizations (AFL-CIO)
The AFL-CIO is the umbrella organization for the American labor
Since that election the Belfast Agreement has continued to offer the best chance for progress in Northern Ireland. This Agreeme

In [3]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ".
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: summit define',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments."]

tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-base-v2')
model = AutoModel.from_pretrained('intfloat/e5-base-v2')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

/cluster/home/sravfogel/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ProxyError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /intfloat/e5-base-v2/resolve/main/tokenizer_config.json (Caused by ProxyError('Unable to connect to proxy', OSError('Tunnel connection failed: 403 Forbidden')))"), '(Request ID: a8ab5936-71d3-4fe1-adc1-83b965356e99)')

In [32]:
i=72
data["original"][i], data["counter"][i]

('Solid Images were commissioned to create a new 3D animated logo for a international coffee company. The company wanted a logo that would be timeless, memorable',
 {'tokens': array([128000,  47041,  12041,   1051,  44224,    311,   1893,    264,
          13166,   2835,  67908,    279,    502,  26097,    315,    264,
          69026,  27767,     13,    578,   2835,   4460,    311,    387,
           5043,  65319,    323,  45955,     11,  39686,    279,  18699,
           4519,    323,  16358,  18112]),
  'count_text': 'Solid Images were commissioned to create a corporate video showcasing the new headquarters of a multinational corporation. The video needed to be fast-paced and energetic, highlighting the innovative features and impressive architecture'})

In [9]:
import os

directory = os.fsencode("counterfactuals")
orig_item = "\\item \\textbf{original}:"
count_item = "\\textbf{counterfactual}:"

for file in os.listdir(directory):
    filename = str(os.fsdecode(file))
    if "bios" in filename and "8B" in filename: continue
    with open("counterfactuals" + "/" + filename, "rb") as f:
        data = pickle.load(f)

    original = data["original"]
    counter = data["counter"]
    print(filename)
    for o,c in zip(original[:5],counter[:5]):
        print(orig_item + o["text"] + "\\\\")
        c_text = c["text"]  if type(c) != list else c[0]["text"]
        print(count_item + c_text)
        print()
    print("=================================")
        

bios_gpt2-xl->mimic_gender_gpt2_prompt:first_k_sents:500_prompt_first_k:8_max_new_tokens:40.pkl
\item \textbf{original}:Tomas Norton is a tenure-track assistant professor in the Center for Education Policy at the University of Maryland, with a focus on school accountability and student outcomes. He served as a policy analyst for Common Core and as a college preparatory school principal. He\\
\textbf{counterfactual}:Tomas Norton is a tenure-track assistant professor in the Departments of Political Science and Sociology at the University of Illinois at Chicago. He is the author of "The Unwinding of American Democracy: How Political Parties Became Polarized and How the

\item \textbf{original}:Xiangyang Xu is an Associate Professor of Huazhong University of Science and Technology, a senior Fellow at the Chinese Academy of Sciences, and the Founding Director of the Huazhong China Global Exchange. He works in China, Thailand, and Vietnam where\\
\textbf{counterfactual}:Xiangyang Xu is an As

In [5]:
len(c)

5